In [82]:
from scipy import integrate
import sympy as sp
import numpy as np


def arc(t0,arc_length) -> float:
    '''
    q4重写方法，只用于计算路径上龙头前进的辐角
    输入当前的角度t0和弧长arc_length
    返回前进到的角度t1
    积分是从t1到t0，因为t1的角度更小,前进过程中角度是在减小的
    '''
    if   arc_length <= 1e4 : #处于盘入螺线上 
        distance_value = 170
        c = distance_value / (2 * sp.pi) # 常数 c，用来计算螺线坐标
        a =  0.5 * c * (sp.sqrt(1 + t0**2) * t0 + sp.asinh(t0))
        t1 = sp.symbols('t1') # 定义符号变量t1,设定为未知数
        b =  0.5 * c * (sp.sqrt(1 + t1**2) * t1 + sp.asinh(t1))
        
        equation = a -b - arc_length
        #sp.pprint(equation)
        result = sp.nsolve(equation,t1, t0)
        
        return float(result)
    
    if 1e4 < arc_length <= 1e4+107.0685 : #
        arc_length = arc_length - 1e4
        distance_value = -170
        c = distance_value / (2 * sp.pi)
        c = distance_value / (2 * sp.pi) # 常数 c，用来计算坐标
        a =  0.5 * c * (sp.sqrt(1 + 0**2) * 0 + sp.asinh(0))
        t1 = sp.symbols('t1') # 定义符号变量t1,设定为未知数
        b =  0.5 * c * (sp.sqrt(1 + t1**2) * t1 + sp.asinh(t1))
        equation = a -b - arc_length
        result = sp.nsolve(equation,t1, t0)
        
        return float(result)
    
    if 1e4+107.0685 < arc_length <= 1e4*2 :   #处于盘出螺线上
        
        
        
        
        
        return 0
        

class Queue:
    '''
    实现一个队列，先入先出(FIFO)
    '''
    def __init__(self):
        self.queue = []
    
    def is_empty(self): 
        return len(self.queue) == 0
    
    def push(self, element):
        self.queue.append(element)
    
    def pop(self):
        if self.is_empty():
            return None
        else:
            return self.queue.pop(0)
    
    def get_size(self):
        return len(self.queue)

'''
在-100s时刻，龙头的位置为32.67299999997398 
在0s时刻，龙头的位置为16.63196110724008 开始进入掉头圆弧第一节

'''
def compute_position(theta):
    '''
    存储曲线数据
    根据输入的辐角theta，返回对应的坐标
    '''
    #if  16.63196110724008 <= theta <=  32.67299999997398: #处于盘入螺线上
    if 0 <= theta : #处于盘入螺线上
        distance_value = 170
        c = distance_value / (2 * np.pi)  
        x = c * theta * np.cos(theta)
        y = c * theta * np.sin(theta)
        return x, y
    
    '''
    if  ... <= theta <= ...:  #处于掉头圆弧上
        distance_value = 170
        c = distance_value / (2 * np.pi)  
        x = c * theta * np.cos(theta)
        y = c * theta * np.sin(theta)
        return x, y
    '''
    
    #if  ... <= theta <= ...: #处于盘出螺线上
    if  theta < 0  : # 此时theta为负值，已经处于盘出螺线上
        theta = -theta  # 将负值转换为正值
        distance_value = -170
        c = distance_value / (2 * np.pi)  
        x = c * theta * np.cos(theta)
        y = c * theta * np.sin(theta)
        return x, y
    
    #return 0, 0
        
def search_vertex(head_theta) -> list:
    '''
    此函数内未用到外部实现的distance函数，采用了内部实现的方法
    '''
    
    theta_step = 0.0005     # 角度步长
    vertex = Queue()      # 队列，用于存储待处理的节点
    vertex_list = []      # 存储所有找到的顶点角度
    
    # 初始角度作为第一个顶点
    vertex.push(head_theta)
    vertex_list.append(head_theta)
    while not vertex.is_empty():
        if len(vertex_list) > 223: # 限制最多只能找到 224 个顶点
            #print("找完了",len(vertex_list))
            break
        
        # 从队列中取出当前的角度（作为线段的固定端点）
        current_theta = vertex.pop()
        #print(f"Processing theta: {current_theta}")
        
        # 当前点的坐标
        x0, y0 = compute_position(current_theta)
        
        # 从当前 theta 开始，移动下一个点
        theta = current_theta
        #print("test",theta)
        while theta <= 32 * np.pi:
            # 尝试计算下一个点的角度

            next_theta = theta + theta_step
                
            x1, y1 = compute_position(next_theta)
            
            # 计算两点之间的距离
            distance = np.sqrt((x1 - x0) ** 2 + (y1 - y0) ** 2)
            if distance > 890:
                print(f"Warning!!!!! Distance too large: {distance}")
            
            # 处理距离为 286 的情况，且这是寻找“头部”的条件
            if distance >= 286 and len(vertex_list) == 1:
                #print("ok")
                #print(f"Found head: {next_theta} with distance: {distance}")
                vertex_list.append(next_theta)
                vertex.push(next_theta)  # 将新找到的顶点作为新的固定端点继续搜索
                break  # 找到符合条件的点后跳出当前循环，开始处理下一个顶点
            
            # 处理距离为 165 的情况
            if distance >= 165 and len(vertex_list) > 1 :
                #print(f"Found vertex at theta: {next_theta} with distance: {distance}")
                vertex_list.append(next_theta)
                vertex.push(next_theta)  # 将新找到的顶点作为新的固定端点继续搜索
                break  # 找到符合条件的点后跳出当前循环，开始处理下一个顶点
            
            # 更新 theta，继续向下搜索
            theta += theta_step
    return vertex_list


def get_speed(t0,time) -> list:
    '''
    输入初始角度和运动时间time
    确定时间微元dt为1e-7
    返回进入螺线的所有节点的坐标和速度
    这里重写了distance函数，因为这里的计算条件不同
    '''
    dt = 1e-4
    current_arc = time*100 #当前头部的弧长
    updated_arc = (time+dt)*100 #dt时间内头部加上走过的弧长
    
    head_theta = arc(t0,current_arc) #头部当前的角度
    updated_head_theta = arc(t0,updated_arc) #dt后头部的角度
    #print(head_theta,updated_head_theta)
    current_theta_list = search_vertex(head_theta)
    #print(current_theta_list)
    after_theta_list = search_vertex(updated_head_theta)  
    #print(after_theta_list)
    
    speed_list = []
    for i in range(len(current_theta_list)):
        a = current_theta_list[i]
        b = after_theta_list[i]
        
        x0, y0 = compute_position(a)
        x1, y1 = compute_position(b)
        ds = np.sqrt((x1 - x0) ** 2 + (y1 - y0) ** 2)
        speed = ds/dt
        speed_list.append(speed)
        
    return current_theta_list,speed_list
'''
for i in range(90,200):
    _,speed = get_speed(32,i)
    print(_[0],i)
'''
_,speed = get_speed(32,200)
print(speed)


[100.00000002473227, 97.57973601330643, 96.1640410688925, 94.72712701464175, 93.26899863047652, 91.78729800254254, 90.28203110546492, 88.7508415788232, 87.19137416922284, 85.60363753189584, 83.98527872850454, 82.33630879808688, 80.65201565653875, 78.93241406831311, 77.17515880772022, 75.3755460159648, 73.53123763981354, 71.63989996174479, 69.6968436874804, 67.69738716028277, 65.63685817405467, 63.50823665748007, 61.30451938993059, 59.016366261452745, 56.63682624094805, 54.149558079407676, 51.540644094901964, 48.78919241880578, 45.86975388462629, 42.745364745737824, 39.36540964844599, 35.654328207896434, 31.48462420926554, 26.61736567881737, 20.456350055216703, 15.171075215313893, 23.151675071247737, 28.673553971755158, 33.22148864002221, 37.18873610652239, 40.756423375245625, 44.02727968310131, 47.06556251228487, 49.915051779926145, 52.60576722765663, 55.163228486709215, 57.60601569380329, 59.94807624606106, 62.20105415716926, 64.37427265849122, 66.47472233675474, 68.50941345102665, 70

In [76]:
#求解q4的准备工作

#求出螺线与掉头空间的两个交点的辐角
def cross_point(radius_value):
    '''
    输入螺线间距值、开始掉头区域的半径值
    输出两个交点的辐角
    '''
    c1 = 170 / (2 * np.pi) #盘入螺线参数
    c2 = -170 / (2 * np.pi) #盘出螺线参数
    
    theta1 = radius_value/c1
    theta2 = radius_value/c2
    return theta1,theta2

def find_minus_100(head_speed,raduis_value):
    
    '''
    输入头部的速度，单位为cm/s
    找到-100s时刻龙头的位置
    '''
    
    def arc_back(t0,arc_length) -> float:
        '''
        输入当前的角度t0和弧长arc_length
        用于临时寻找-100s时刻龙头的位置
        这里不需要重写条件，直接借用原来的条件
        '''
        distance_value = -170
        c = distance_value / (2 * sp.pi) # 常数 c，用来计算螺线坐标
        a =  0.5 * c * (sp.sqrt(1 + t0**2) * t0 + sp.asinh(t0))
        t1 = sp.symbols('t1') # 定义符号变量t1,设定为未知数
        b =  0.5 * c * (sp.sqrt(1 + t1**2) * t1 + sp.asinh(t1))
        equation = a -b - arc_length
        result = sp.nsolve(equation,t1, t0)
        return float(result)
    
    head_v = head_speed
    arc_length = 100*head_v
    
    #以交点1为起点
    t0 = cross_point(raduis_value)[0]
    minus_100 = arc_back(t0,arc_length)
    
    return minus_100
 
'''   
start = find_minus_100(100,450)
print("start:",start)
get_speed(start,137.9)
'''
print(cross_point(301))

(11.124933985065033, -11.124933985065033)


In [86]:
import sympy as sp
import numpy as np
import pandas as pd

def q4_posi():
    def main_q4(time):
        theta_list , speed_list = get_speed(32,time)
        print(len(theta_list),theta_list)
        print(len(speed_list),speed_list)
        x_list = []
        y_list = []
        target_speed = []
        for i in range(len(theta_list)):
                temp = theta_list[i]
                x = 55 / (2 * np.pi) * temp * np.cos(temp)
                y = 55 / (2 * np.pi) * temp * np.sin(temp)
                print(f"theta: {temp}, x: {x}, y: {y}")
                x_list.append(x)
                y_list.append(y)
                target_speed.append(speed_list[i])
                
        print("x:",x_list)
        print("y:",y_list)

        df = pd.DataFrame({"x":[o/100 for o in x_list],"y":[p/100 for p in y_list]})
        
        x = df["x"].tolist()
        y = df["y"].tolist()
        
        alternating_list = [val for pair in zip(x, y) for val in pair]
        df_alternating = pd.DataFrame({time:alternating_list})
        df_alternating.to_csv(f"{time}.csv",index=True)
        print(df_alternating)

    for i in range(1,200+2):
        main_q4(i)
        print("time:",i)
        print("--------------------------------------------------")


def q4_speed():
    def main_q4(time):
        theta_list , speed_list = get_speed(32,time)
        df = pd.DataFrame({f"{time}":[o/100 for o in speed_list]})
        return df
    for i in range(1,200+2):
        df = main_q4(i)
        df.to_csv(f"{i}.csv",index=True)
        print("time:",i)

q4_speed()


time: 1
time: 2
time: 3
time: 4
time: 5
time: 6
time: 7
time: 8
time: 9
time: 10
time: 11
time: 12
time: 13
time: 14
time: 15
time: 16
time: 17
time: 18
time: 19
time: 20
time: 21
time: 22
time: 23
time: 24
time: 25
time: 26
time: 27
time: 28
time: 29
time: 30
time: 31
time: 32
time: 33
time: 34
time: 35
time: 36
time: 37
time: 38
time: 39
time: 40
time: 41
time: 42
time: 43
time: 44
time: 45
time: 46
time: 47
time: 48
time: 49
time: 50
time: 51
time: 52
time: 53
time: 54
time: 55
time: 56
time: 57
time: 58
time: 59
time: 60
time: 61
time: 62
time: 63
time: 64
time: 65
time: 66
time: 67
time: 68
time: 69
time: 70
time: 71
time: 72
time: 73
time: 74
time: 75
time: 76
time: 77
time: 78
time: 79
time: 80
time: 81
time: 82
time: 83
time: 84
time: 85
time: 86
time: 87
time: 88
time: 89
time: 90
time: 91
time: 92
time: 93
time: 94
time: 95
time: 96
time: 97
time: 98
time: 99
time: 100
time: 101
time: 102
time: 103
time: 104
time: 105
time: 106
time: 107
time: 108
time: 109
time: 110
time: 11